In [27]:
# Python 3.x
import re
import math

def Input(day):
    "Open this day's input file."
    filename = 'inputs/{}.txt'.format(day)
    try:
        return open(filename)
    except FileNotFoundError:
        print("Oops, couldn't open input")


# Day 1 - Inverse Captcha
## Part 1

> The captcha requires you to review a sequence of digits (your puzzle input) and find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

This could be done by reading the digits into an array, then reducing them. As a Python noob, my reading tells me that for clarity, list comprehensions are preferred over lambdas, e.g. see Guido van Rossum's [post](http://www.artima.com/weblogs/viewpost.jsp?thread=98196) from 2005. However, list comprehensions produce lists, whereas reduce produces a single output, and since a sum is computed using addition, which is associative, I'm going to use `reduce()`.

In [4]:
from functools import reduce

In [31]:
def circular_list_next_item(list, index): 
    "Returns the item next to the item given by the index; the item after the last item is the first item"
    return list[0] if index == len(list)-1 else list[index+1]

def inverse_captcha(digits):
    "Returns sum of digits in list that match the next digit, treating list as circular."
    return reduce(lambda sum, index: sum + (digits[index] if circular_list_next_item(digits, index) == digits[index] else 0), range(len(digits)), 0)
    
def characters_to_int_array(input):
    "Given a string of digits, returns them as an array of integers"
    return [int(x) for x in input]

assert inverse_captcha(characters_to_int_array("1122")) == 3
assert inverse_captcha(characters_to_int_array("1111")) == 4
assert inverse_captcha(characters_to_int_array("1234")) == 0
assert inverse_captcha(characters_to_int_array("91212129")) == 9

inverse_captcha(characters_to_int_array(Input(1).read().rstrip("\n")))

1141